# Regresión: 2 neuronas con varias columnas en la X

La estructura es la siguiente:

1. **Capa oculta** (Neurona 1):
   - Entrada: $\mathbf{x}\in\mathbb{R}^2$ (dos características).
   - Parámetros: $\mathbf{w}_1 \in \mathbb{R}^2$ y $b_1 \in \mathbb{R}$.
   - Salida lineal: $z_1 = \mathbf{x}\cdot\mathbf{w}_1 + b_1$.
   - Activación ReLU: $a_1 = \max(0, z_1)$.

2. **Capa de salida** (Neurona 2):
   - Entrada: $a_1\in\mathbb{R}$ (escalar).
   - Parámetros: $w_2 \in \mathbb{R}$ (un solo número) y $b_2 \in \mathbb{R}$.
   - Salida lineal: $z_2 = a_1 \cdot w_2 + b_2$.
   - Sin activación adicional (la salida es $y_{\text{pred}} = z_2$).

In [2]:
import numpy as np

# X: shape (N, 2)
# - 1ª columna: años de experiencia
# - 2ª columna: nivel de estudios (codificado numéricamente)
X = np.array([
    [1, 0],   # persona1
    [2, 0],   # persona2
    [3, 1],   # persona3
    [5, 1],   # persona4
    [6, 2],   # persona5
], dtype=float)

# y: salarios (en miles, por ejemplo)
y = np.array([20, 25, 35, 45, 60], dtype=float)

N = X.shape[0]  # Número de ejemplos (N=5)

## Forward pass

Función de activación ReLU:

$$
\text{ReLU}(z) = \max(0, z).
$$

1. **Capa oculta**:  
   - $ z_1^{(i)} = X^{(i)} \cdot w_1 + b_1 $  
   - $ a_1^{(i)} = \text{ReLU}(z_1^{(i)}) $

2. **Capa de salida**:  
   - $ z_2^{(i)} = a_1^{(i)} \cdot w_2 + b_2 $  
   - Predicción: $ y_{\text{pred}}^{(i)} = z_2^{(i)} $

> Devolvemos `z1` y `a1` porque los necesitaremos en el **backprop** (cálculo de gradientes).

In [3]:
def relu(z):
    return np.maximum(0, z)

def forward(X, w1, b1, w2, b2):
    # Capa oculta
    z1 = np.dot(X, w1) + b1   # (N,)
    a1 = relu(z1)            # (N,)  ReLU

    # Capa de salida (lineal)
    z2 = a1 * w2 + b2        # (N,)
    y_pred = z2              # (N,)  (sin activación final)

    return y_pred, z1, a1

## Función coste

In [4]:
def mse_loss(y_pred, y_true):
    return np.mean((y_pred - y_true)**2)

## Gradientes

- `X`: entradas con dos columnas (**shape (N, 2)**).
- `w1`: pesos de la primera capa (**shape (2,)**).
- `b1`: sesgo de la primera capa (**escalar**).
- `w2`: peso de la segunda capa (**escalar**).
- `b2`: sesgo de la segunda capa (**escalar**).
- `forward(X, w1, b1, w2, b2)`: función que realiza el forward pass.

#### Cálculo del gradiente para `w2` y `b2` (segunda capa)

- **`grad_w2`:**  
$$
\frac{\partial \text{Loss}}{\partial w_2} = \frac{2}{N} \sum (y_{\text{pred}} - y) \cdot a_1
$$

- **`grad_b2`:**  
$$
\frac{\partial \text{Loss}}{\partial b_2} = \frac{2}{N} \sum (y_{\text{pred}} - y)
$$


#### Cálculo del gradiente para `w1` y `b1` (primera capa)

Aquí necesitamos aplicar la **regla de la cadena**. La derivada se ve afectada por la función ReLU, cuya derivada es:

$$
\text{ReLU}'(z) = \begin{cases} 
1 & \text{si } z > 0 \\
0 & \text{si } z \leq 0
\end{cases}
$$

La derivada de la pérdida respecto a `w1` es:

$$
\frac{\partial \text{Loss}}{\partial w_1} = \frac{2}{N} \sum \left( (y_{\text{pred}} - y) \cdot w_2 \cdot \text{ReLU}'(z_1) \cdot X \right)
$$

#### Explicación de los pasos

1. **Forward pass**: Calcula `z1`, `a1` y `y_pred`.
2. **Backward pass**:
   - Para `w2` y `b2`, el gradiente se basa en el error directo entre `y_pred` y `y`.
   - Para `w1` y `b1`, aplicamos la regla de la cadena. Multiplicamos el error por `w2` y la derivada de la función **ReLU** para propagar los gradientes hacia atrás.

Por tanto:

- **`grad_w2`** y **`grad_b2`** se calculan directamente a partir de la salida `a1`.
- **`grad_w1`** y **`grad_b1`** usan la regla de la cadena, multiplicando el error por `w2` y la derivada de **ReLU**.

In [5]:
def grad_w2(y_pred, y, a1):
    """
    Calcula el gradiente respecto a w2 (escalar).
    """
    N = len(y)
    return (2.0 / N) * np.dot(a1, (y_pred - y))

def grad_b2(y_pred, y):
    """
    Calcula el gradiente respecto a b2 (escalar).
    """
    N = len(y)
    return (2.0 / N) * np.sum(y_pred - y)

In [6]:
def grad_w1(X, y, w1, b1, w2, b2):
    """
    Calcula el gradiente respecto a w1 (vector de forma (2,)).
    """
    N = len(X)

    # Forward pass
    z1 = np.dot(X, w1) + b1        # shape (N,)
    a1 = relu(z1)                  # shape (N,)
    y_pred = a1 * w2 + b2          # shape (N,)

    # Backward pass
    error = (y_pred - y)           # shape (N,)
    relu_derivative = (z1 > 0).astype(float)  # Derivada de ReLU: 1 donde z1 > 0, 0 en caso contrario

    # Gradiente respecto a w1
    return (2.0 / N) * np.dot(X.T, error * w2 * relu_derivative)

def grad_b1(X, y, w1, b1, w2, b2):
    """
    Calcula el gradiente respecto a b1 (escalar).
    """
    N = len(X)

    # Forward pass
    z1 = np.dot(X, w1) + b1        # shape (N,)
    a1 = relu(z1)                  # shape (N,)
    y_pred = a1 * w2 + b2          # shape (N,)

    # Backward pass
    error = (y_pred - y)           # shape (N,)
    relu_derivative = (z1 > 0).astype(float)  # Derivada de ReLU

    # Gradiente respecto a b1
    return (2.0 / N) * np.sum(error * w2 * relu_derivative)

In [7]:
# alternativa combinada en una sola función:

def backward(X, y, w1, b1, w2, b2):
    """
    Calcula los gradientes de la MSE wrt w1, b1, w2, b2.
    Retorna dw1, db1, dw2, db2.
    """
    N = len(X)
    
    # 1. Forward (para obtener z1, a1, z2)
    y_pred, z1, a1 = forward(X, w1, b1, w2, b2)
    
    # 2. r = y_pred - y  (residuo)
    r = (y_pred - y)  # shape (N,)

    # 3. Gradientes de la neurona de salida (z2)
    # dw2 = (2/N)*sum((z2-y)*a1)
    # db2 = (2/N)*sum((z2-y))
    dw2 = (2.0 / N) * np.sum(r * a1)
    db2 = (2.0 / N) * np.sum(r)
    
    # 4. Gradientes de la capa oculta (z1 -> a1 -> z2)
    # dMSE/dz1 = (2.0/N)*r * w2 * ReLU'(z1)
    # ReLU'(z1) = 1 si z1>0, 0 si z1<=0
    relu_mask = (z1 > 0).astype(float)  # shape (N,)
    dz1 = (2.0 / N) * r * w2 * relu_mask  # shape (N,)

    # dw1 = sum(X.T * dz1)
    #   x^{(i)} -> shape(2,)
    #   dz1 -> shape(N,)
    #   X: shape(N,2), X.T: shape(2,N)
    dw1 = np.dot(X.T, dz1)  # shape(2,)
    
    # db1 = sum(dz1)  (derivada wrt b1)
    db1 = np.sum(dz1)
    
    return dw1, db1, dw2, db2

## Entrenamiento

Al finalizar, tendremos valores de $\mathbf{w}_1, b_1, w_2, b_2$ que ajustan la red a los datos. 

In [ ]:
np.random.seed(42)
w1 = np.random.randn(2) * 0.01  # vector (2,)
b1 = 0.0

w2 = np.random.randn() * 0.01   # escalar
b2 = 0.0

learning_rate = 0.001
num_epochs = 5000

for epoch in range(num_epochs):
    # forward
    y_pred, z1, a1 = forward(X, w1, b1, w2, b2)
    loss_value = mse_loss(y_pred, y)
    
    # backward
    dw1, db1, dw2, db2 = backward(X, y, w1, b1, w2, b2)
    
    # Actualizar parámetros
    w1 -= learning_rate * dw1
    b1 -= learning_rate * db1
    w2 -= learning_rate * dw2
    b2 -= learning_rate * db2
    
    # (Opcional) Print each certain steps
    if (epoch+1) % 1000 == 0:
        print(f"Epoch {epoch+1}/{num_epochs} - Loss: {loss_value:.4f}",
              f"w1:{w1}, b1:{b1:.4f}, w2:{w2:.4f}, b2:{b2:.4f}")

Epoch 1000/5000 - Loss: 1.7296 w1:[1.48814629 1.56701076], b1:3.0149, w2:3.7072, b2:2.3781
Epoch 2000/5000 - Loss: 1.3933 w1:[1.23778612 1.94325172], b1:3.1338, w2:3.8875, b2:2.4094
Epoch 3000/5000 - Loss: 1.3889 w1:[1.21023368 1.98411757], b1:3.1458, w2:3.9091, b2:2.4125
Epoch 4000/5000 - Loss: 1.3889 w1:[1.20757056 1.98806205], b1:3.1469, w2:3.9112, b2:2.4128
Epoch 5000/5000 - Loss: 1.3889 w1:[1.20731668 1.98843803], b1:3.1470, w2:3.9114, b2:2.4128


In [9]:
print("\nEntrenamiento finalizado.")
print(f"Parámetros: w1={w1}, b1={b1}, w2={w2}, b2={b2}")

y_pred_final, _, _ = forward(X, w1, b1, w2, b2)
print("Predicciones finales:", y_pred_final)
print("Reales              :", y)


Entrenamiento finalizado.
Parámetros: w1=[1.20731668 1.98843803], b1=3.1470202572665587, w2=3.911402265486647, b2=2.4128455209100226
Predicciones finales: [19.44440888 24.16671007 36.66659227 46.11119465 58.61107685]
Reales              : [20. 25. 35. 45. 60.]


In [10]:
X_nuevo = np.array([[4, 1.5]], dtype=float)
y_nuevo_pred, _, _ = forward(X_nuevo, w1, b1, w2, b2)
print("Predicción para X=[4,1.5]:", y_nuevo_pred)

Predicción para X=[4,1.5]: [45.27768396]
